# Testing and Inference with the Emotion-Aware Chatbot

This notebook demonstrates the testing and inference pipeline for an emotion-aware chatbot. The chatbot uses a fine-tuned DialoGPT model to generate responses and a fine-tuned DistilBERT model to detect the user's emotion, which is used to condition the response.

## Setup

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
%cd ../

/


In [12]:
!pip install torch
!pip install transformers
!pip install datasets

!pip install accelerate -U
!pip install nltk
!python -m nltk.downloader punkt

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, DistilBertForSequenceClassification
from datasets import load_dataset

<frozen runpy>:128: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Load the Models and Tokenizer

In [13]:
import os
path = "/content/drive/MyDrive/distilbert-emotion"
print("files:", os.listdir(path))

files: ['tokenizer_config.json', 'special_tokens_map.json', 'config.json', 'vocab.txt', 'model.safetensors']


In [14]:
tokenizer = AutoTokenizer.from_pretrained('microsoft/DialoGPT-small', padding_side='left')
model = AutoModelForCausalLM.from_pretrained('/content/drive/MyDrive/DialoGPT-chatbot-emotion-aware')


from transformers import pipeline
import torch

# Define device at the beginning
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Folder containing your trained DistilBERT emotion model
emotion_model_dir = "/content/drive/MyDrive/distilbert-emotion"

# Load tokenizer
emotion_tokenizer = AutoTokenizer.from_pretrained(emotion_model_dir)

# Load model (trained) from safetensors
emotion_model = DistilBertForSequenceClassification.from_pretrained(
    emotion_model_dir,
    device_map="auto"  # puts on GPU if available
)


from torch.nn.functional import softmax

emotion_dict = {0: 'neutral', 1: 'anger', 2: 'disgust', 3: 'fear', 4: 'happiness', 5: 'sadness', 6: 'surprise'}

## Evaluate on Test Dataset

In [15]:
!pip install datasets==3.6.0

In [16]:
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from datasets import load_dataset
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load trained DialoGPT
model_path = '/content/drive/MyDrive/DialoGPT-chatbot-emotion-aware'
tokenizer = AutoTokenizer.from_pretrained('microsoft/DialoGPT-small', padding_side='left')
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(model_path).to(device)

# Load test dataset
dataset = load_dataset("daily_dialog")
test_dataset = dataset['test']

emotion_dict = {0: 'neutral', 1: 'anger', 2: 'disgust', 3: 'fear', 4: 'happiness', 5: 'sadness', 6: 'surprise'}

# Preprocess test dataset the same way as for training
def encode_with_emotion(examples):
    dialog_with_emotions = []
    for dialog, emotion in zip(examples['dialog'], examples['emotion']):
        dialog_with_emotion = [f"{emotion_dict[emotion[i]]}: {dialog[i]}" for i in range(len(dialog))]
        dialog_with_emotions.append(" ".join(dialog_with_emotion))
    encoded = tokenizer(dialog_with_emotions, truncation=True, padding='max_length', max_length=128)
    encoded['labels'] = encoded['input_ids'][:]
    return encoded

test_dataset = test_dataset.map(encode_with_emotion, batched=True)

# Evaluate
training_args = TrainingArguments(
    output_dir="./eval",
    per_device_eval_batch_size=8,
    prediction_loss_only=True,
    report_to=[]
)

trainer = Trainer(
    model=model,
    args=training_args,
    eval_dataset=test_dataset
)

eval_results = trainer.evaluate()
print("Test Loss:", eval_results['eval_loss'])
print("Perplexity:", torch.exp(torch.tensor(eval_results['eval_loss'])))


Test Loss: 5.625612258911133
Perplexity: tensor(277.4421)


**Insight:**  
The model shows a loss of 5.63 on the test set, with a corresponding perplexity of about 277. This suggests that while the chatbot can predict the next word reasonably well, there is still uncertainty in its predictions.

In practical terms, the model is capable of generating coherent responses, but some outputs may not always feel perfectly fluent

these results are in line with expectations for a model trained on a conversational dataset like DailyDialog.

## Interactive Chat with Emotion Detection

In [19]:
model = model.to(device)
emotion_model = emotion_model.to(device)

emotion_model.eval()

for step in range(10):

    user_message = input(">> User:")

    # Prepare the message for input to the emotion model
    encoded_input = emotion_tokenizer(user_message, return_tensors='pt').to(device)

    # Get the detected emotion of the user's message
    with torch.no_grad():
        emotion_outputs = emotion_model(**encoded_input)

    # Get the predicted emotion label
    predicted_emotion_id = torch.argmax(emotion_outputs.logits, dim=1).item()
    detected_emotion = emotion_dict[predicted_emotion_id]

    # Prepend the detected emotion to the user's message
    user_message = f"{detected_emotion}: {user_message}"

    # Encode the new user input, add the eos_token and return a tensor in Pytorch
    new_user_input_ids = tokenizer.encode(user_message + tokenizer.eos_token, return_tensors='pt')

    # Only use the last user input
    bot_input_ids = new_user_input_ids

    # Generate a response
    chat_history_ids = model.generate(bot_input_ids.to(device), max_length=50, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id, do_sample=True, temperature=0.7)

    # Decode the output and remove special tokens
    output_text = tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)

    # Split the output into sentences
    sentences = output_text.split('.')

    # Print only the first sentence
    print(f"DialoGPT: {sentences[0]}")

    # # Pretty print last output tokens from bot
    # print("DialoGPT: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_token=True)))

>> User:i am happy to meet you
DialoGPT: It's nice to meet you, too
>> User:what's your name?
DialoGPT: My name is Helen
>> User:where are you from?
DialoGPT: San Francisco
>> User:are you excited for the movie?
DialoGPT: No
>> User:are you happy to go to the park?
DialoGPT: Yes, I am happy to go to the park
>> User:i am happy too
DialoGPT: We are very happy for you
>> User:i am worried because of my exam
DialoGPT: Why are you worried?
>> User:because it is stressful
DialoGPT: Why don't we go to the cinema?
>> User:let's go
